# Building custom components

In NVFlare, the [`FLComponent`](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.apis.fl_component.html#nvflare.apis.fl_component.FLComponent) is the base class of all components, including controllers, executors, responders, filters, aggregators and many others (see the [here](https://nvflare.readthedocs.io/en/main/programming_guide/fl_component.html) for more details). `FLComponent`s have the capability to handle and fire events and contain various methods for logging.

In this notebook, we'll explore how to implement a very basic custom `Controller` and `Executor` in NVFlare:
- a [`Controller`](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.apis.impl.controller.html) is a server-side component responsible for managing job execution and orchestrating tasks across clients
- an [`Executor`](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.apis.executor.html) is a client-side component that processes tasks received from the controller and executes them accordingly

For this first tutorial we won't go into the details of tasks and execution. Instead, we'll start by creating placeholder versions of these components and then add some very basic functionality to them to better understand the flow of execution and the roles of the different components involved.

## The Controller
We'll start by creating a `PlaceholderController` class that doesn't perform any specific tasks but serves as a template for understanding how controllers work in NVFlare.

As a subclass of `Controller`, our `PlaceholderController` must implement three methods:

- `control_flow`: defining the main control flow of the controller. It receives an `abort_signal: Signal` and an `fl_ctx: FLContext`. A [`Signal`](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.apis.signal.html#nvflare.apis.signal.Signal) is an object that provides a mechanism to signal events like abortion. Controllers and executors can check this signal to determine if they should stop execution gracefully. More details on handling abort signals [here](https://nvflare.readthedocs.io/en/main/best_practices.html#respect-the-abort-signal). An [`FLContext`](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.apis.fl_context.html#nvflare.apis.fl_context.FLContext) object carries all the execution context, which includes information about the current execution environment, such as run number, job ID, and other configurations. It's passed to many methods to provide context. More details can be found [here](https://nvflare.readthedocs.io/en/main/programming_guide/fl_context.html). We'll talk more about it in the next notebooks.
- `start_controller`: called once before the `control_flow` method. It's used to perform any setup tasks or initialize resources.
- `stop_controller`: called once after the `control_flow` method. It's used to clean up resources or perform any finalization tasks.

For the moment, we'll leave these methods empty. We'll add some functionality to them later in the notebook.

```python
from nvflare.apis.fl_context import FLContext
from nvflare.apis.impl.controller import Controller
from nvflare.apis.signal import Signal

class PlaceholderController(Controller):

    def control_flow(self, abort_signal: Signal, fl_ctx: FLContext):
        pass

    def start_controller(self, fl_ctx: FLContext):
        pass

    def stop_controller(self, fl_ctx: FLContext):
        pass
```

Let's try to instantiate the controller and run it via the NVFlare simulator. As we've seen in previous chapters, objects sent to server and clients need to be importable by the simulator. Since we are in a notebook, we can't import the controller directly, so let's put it in a separate file `modules.py` and import it from there - for convenience, we have already included all the custom components we'll use in this chapter in `modules.py`, but feel free to delete the file and recreate it from scratch while we move forward.

In [ ]:
from nvflare.job_config.api import FedJob
from modules import PlaceholderController

# Create job
job = FedJob(name="placeholder_job")

# send controller to server
controller = PlaceholderController()
job.to_server(controller)

# Run job via the NVFlare simulator
job.simulator_run("./tmp/")

Now, as you can see, trying to run the previous job results in an error. This is because we have only created a server running our `PlaceholderController` but we haven't sent any executors to the clients. 

To fix that, let's move on to the next section to explore `Executor`s and create a custom one.

## The Executor

When subclassing an `Executor` the main method that must be implmented is the `execute` method, which defines how the `Executor` processes a task received from the `Controller`. It receives the following arguments in addition to an `abort_signal` and `fl_ctx`:
- `task_name`: the name of the task to be executed
- `shareable`: a [`Shareable`](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.apis.shareable.html#nvflare.apis.shareable.Shareable) object containing the task data. We'll talk more about it in the next notebook but for the moment just note that this is the data structure used for communication between the server and clients. It wraps the data exchanged and can include metadata or headers. More details on `Shareable`s [here](https://nvflare.readthedocs.io/en/main/programming_guide/shareable.html).

Let's create a `PlaceholderExecutor` class that doesn't perform any specific tasks but serves as a template for understanding how executors work in NVFlare.

```python
class PlaceholderExecutor(Executor):

    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        pass
```

Let's now add 3 clients, assign our `PlaceholderExecutor` to each of them and run our job.

In [ ]:
from modules import PlaceholderExecutor

# Create job
job = FedJob(name="placeholder_job")

# send controller to server
controller = PlaceholderController()
job.to_server(controller)

# send executor to clients
num_clients = 3
for i in range(num_clients):
    executor = PlaceholderExecutor()
    job.to(executor, f"site-{i}")

# Run job via the NVFlare simulator
job.simulator_run("./tmp/")

When you run the above code, you'll notice that the number of `Total clients` in the system increases from 1 to 3 as the clients start. However, since both the controller and executors are not performing any specific operations (their methods are empty), the job completes without any significant actions.

## Logging info

To explore the actual flow of execution, let's add some very basic logging to our `PlaceholderController` and `PlaceholderExecutor`. 

We want both our controller and executors to log when they start and stop but we'll do that in two different ways. 

For the controller, we'll do that in the `start_controller` and `stop_controller` methods.
For the executors, we'll do that by overriding their `handle_event` method, which as the name suggests, is used to handle different events. Notice that NVFlare uses an event-driven architecture where components can fire and handle events to coordinate actions. By checking for `EventType.START_RUN` or `EventType.END_RUN` events we can log messages when the executor starts and stops (you can read more about event handling [here](https://nvflare.readthedocs.io/en/main/programming_guide/component_configuration.html#component-configuration-and-event-handling)).

We can use the `log_info` method which is available to any subclass of `FLComponent` (which both `Controller` and `Executor` inherit from).

```python
class LoggingController(Controller):
    def control_flow(self, abort_signal: Signal, fl_ctx: FLContext):
        pass

    def start_controller(self, fl_ctx: FLContext):
        self.log_info(fl_ctx, "Starting the controller...")

    def stop_controller(self, fl_ctx: FLContext):
        self.log_info(fl_ctx, "Stopping the controller...")


class LoggingExecutor(Executor):
    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        pass

    def handle_event(self, event_type, fl_ctx):
        if event_type == EventType.START_RUN:
            self.log_info(fl_ctx, "Starting the executor...")
        elif event_type == EventType.END_RUN:
            self.log_info(fl_ctx, "Stopping the executor...")
```

In [ ]:
from modules import LoggingController, LoggingExecutor

# Create job
job = FedJob(name="logging_job")

# send controller to server
controller = LoggingController()
job.to_server(controller)

# send executor to clients
num_clients = 3
for i in range(num_clients):
    executor = LoggingExecutor()
    job.to(executor, f"site-{i}")

# Run job via the NVFlare simulator
job.simulator_run("./tmp/")

Now, if you look at the output of the cell above and look for the logs of the `LoggingController` and `LoggingExecutor`s, you should see that they are logging the start and stop events. You should also see that the controller is logging the start and stop events before and after the executors are logging theirs. We'll see more about that in the next section.